In [ ]:
!git clone https://github.com/joren015/ASLRecognition.git

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp -r /content/ASLRecognition/Datasets /content
!cp /content/ASLRecognition/Functions.py /content
!cp /content/ASLRecognition/Model.py /content
!cp /content/ASLRecognition/NotebookUtils.py /content
!mkdir /content/Datasets/bronze
!mkdir /content/Datasets/bronze/homemade_dataset
!cp -r /content/drive/Shareddrives/CSCI5561_EricksonJorenby_CollectedDataset/CSCI5561_EricksonJorenby_CollectedDataset/StillImageDataset.zip /content
!unzip /content/StillImageDataset.zip -d /content/Datasets/bronze/homemade_dataset

In [ ]:
!pip install mediapipe==0.8.0

In [ ]:
from os import walk, makedirs, listdir, cpu_count, environ
from os.path import join, exists, isfile, join
import cv2

if not exists("/content/Datasets/silver/homemade_dataset/"):
    makedirs("/content/Datasets/silver/homemade_dataset/")

count = 0
dirs = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y"]
for i in dirs:
  print(i)
  image_dir = "/content/Datasets/bronze/homemade_dataset/{}/".format(i)
  files = ["{}{}".format(image_dir, x) for x in listdir(image_dir) if x.split('.')[-1] == 'jpg']
  
  for f in files:
    print(f)
    img = cv2.imread(f)
    res = cv2.rotate(img, cv2.cv2.ROTATE_90_CLOCKWISE)
    save_dir = "/content/Datasets/silver/homemade_dataset/center_2_{}_{}.png".format(i, count)
    cv2.imwrite(save_dir, res)
    count+=1

In [ ]:
from NotebookUtils import PrintDatetime
PrintDatetime("Importing packages")
import itertools  
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mediapipe as mp
import shutil
from os import walk, makedirs, listdir, cpu_count, environ
from os.path import join, exists, isfile, join
from sklearn.model_selection import train_test_split
from Functions import *
from Model import *
PrintDatetime()


root_dir = "/content"
dataset_path = "{}/Datasets".format(root_dir)
if not exists(root_dir):
  makedirs(root_dir)

directories = ["{}/models", "{}/Datasets/bronze/homemade_dataset", "{}/Datasets/silver/homemade_dataset", "{}/Datasets/silver/homemade_dataset_resized", "{}/Datasets/gold/Homography", "{}/Datasets/gold/Censure", "{}/Datasets/gold/Transforms", "{}/Datasets/gold/homemade_dataset_resized", "{}/Datasets/gold/homemade_dataset_transformed/"]
directories = [x.format(root_dir) for x in directories]
for d in directories:
  if not exists(d):
    makedirs(d)




train_test_path_base = "{}/gold".format(dataset_path)
alpha_num = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"]
encoding_config = {"a": 0, "b": 1,"c": 2,"d": 3,"e": 4,"f": 5,"g": 6,"h": 7
                   ,"i": 8,"j": 9,"k": 10,"l": 11,"m": 12,"n": 13,"o": 14
                   ,"p": 15,"q": 16,"r": 17,"s": 18,"t": 19,"u": 20,"v": 21
                   ,"w": 22,"x": 23,"y": 24,"z": 25,"0": 26,"1": 27,"2": 28
                   ,"3": 29,"4": 30,"5": 31,"6": 32,"7": 33,"8": 34,"9": 35 }
PrintDatetime()




PrintDatetime("Started")
image_dir = "/content/Datasets/silver/homemade_dataset/"
print(image_dir)
file_list = ["{}{}".format(image_dir, x) for x in listdir(image_dir)]
save_dir = "/content/Datasets/silver/homemade_dataset_annotated/"
landmarks_save_dir = "/content/Datasets/gold/homemade_dataset_resized/"
MPLoop(file_list, save_dir, landmarks_save_dir)

PrintDatetime()




PrintDatetime("Started resizing image dataset")
PreprocessDatasets("{}/silver/homemade_dataset".format(dataset_path), "{}/gold/homemade_dataset_resized".format(dataset_path), ".png", overwrite=True)
PrintDatetime()


In [ ]:
from os import remove

load_dir = "/content/Datasets/gold/homemade_dataset_resized"

dirs = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y"]
for c in dirs:
  image_paths = []
  files_paths = []
  image_paths += ["{}/{}".format(load_dir, x) for x in listdir(load_dir) if x.split('.')[-1] == "png" and x.split("_")[2] == c]
  files_paths += ["{}/{}".format(load_dir, y) for y in listdir(load_dir) if y.split('.')[-1] == "pt" and y.split("_")[2] == c]
  print(image_paths)
  print(files_paths)
  
  for image_i in image_paths:
    landmarks_i = image_i.replace(".png", "_landmarks.pt")
    if landmarks_i not in files_paths:
      rem_pt = image_i.replace(".png", ".pt")
      remove(image_i)
      remove(rem_pt)





PrintDatetime("Started determining new baseline")
baseline_results_path = "{}/bronze/homemade_dataset".format(dataset_path)
DetermineNewBaseline("{}/gold/homemade_dataset_resized".format(dataset_path), baseline_results_path)
PrintDatetime()

In [ ]:
if not exists("/content/Datasets/gold/homemade_dataset_transformed/homography/"):
    makedirs("/content/Datasets/gold/homemade_dataset_transformed/homography/")
if not exists("/content/Datasets/gold/homemade_dataset_transformed/censure/"):
    makedirs("/content/Datasets/gold/homemade_dataset_transformed/censure/")

PrintDatetime("Started transforming video dataset")

dirs = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y"]
results = {c: {} for c in dirs}
transforms = ["homography", "CENSURE"]
for transform in transforms:
  if transform=="homography":
    baseline2 = NewBaseline(baseline_results_path + "/baseline_eval_homography_homemade_dataset.json")
  if transform=="CENSURE":
    baseline2 = NewBaseline(baseline_results_path + "/baseline_eval_CENSURE_homemade_dataset.json")

  for c in dirs:
    image_paths = []
    image_dir = "{}/gold/homemade_dataset_resized".format(dataset_path)
    image_paths += ["{}/{}".format(image_dir, x) for x in listdir(image_dir) if x.split('.')[-1] == "png" and x.split("_")[2] == c]
    baseline_img_path = [x for x in baseline2 if x.split('/')[-1].split('_')[2] == c][0]
    baseline_fp_path = baseline_img_path.replace(".png", "_landmarks.pt")
    baseline_img = cv2.imread(baseline_img_path)
    baseline_fp = GetPoints(baseline_img_path, baseline_fp_path)
    for img_path in image_paths:
      if img_path != baseline_img_path:
        img_fp_path = img_path.replace(".png", "_landmarks.pt")
        img = cv2.imread(img_path)
        img_fp = GetPoints(img_path, img_fp_path)
        if transform == "homography":
          img_transformed = HomographyTransform(img, baseline_img, img_fp, baseline_fp)
          cv2.imwrite(img_path.replace("homemade_dataset_resized", "homemade_dataset_transformed/homography"), img_transformed)
          torch.save(torch.from_numpy(img_transformed.reshape(3, 200, 200)), img_path.replace("homemade_dataset_resized", "homemade_dataset_transformed/homography").replace(".png", ".pt"))
        if transform == "CENSURE":
          img_transformed = CENSURETransform(img, baseline_img, img_fp, baseline_fp)
          #cv2.imwrite(img_path.replace("homemade_dataset_resized", "homemade_dataset_transformed/censure"), img_transformed)
          #torch.save(torch.from_numpy(img_transformed.reshape(3, 200, 200)), img_path.replace("homemade_dataset_resized", "homemade_dataset_transformed/censure").replace(".png", ".pt"))
        

    print(image_paths)
    print("Character: {}".format(c))

PrintDatetime()

In [ ]:
PrintDatetime("Started labeling non-transformed image data")
LabelDatasets(load_path="{}/gold/homemade_dataset_resized".format(dataset_path), save_path="{}/gold/homemade_dataset_resized".format(dataset_path), encoding_config=encoding_config)
PrintDatetime()



PrintDatetime("Started labeling annotations transformed image data")
LabelDatasets(load_path="{}/gold/homemade_dataset_transformed/homography".format(dataset_path), save_path="{}/gold/homemade_dataset_transformed/homography".format(dataset_path), encoding_config=encoding_config)
PrintDatetime()



PrintDatetime("Started labeling CENSURE transformed image data")
LabelDatasets(load_path="{}/gold/homemade_dataset_transformed/censure".format(dataset_path), save_path="{}/gold/homemade_dataset_transformed/censure".format(dataset_path), encoding_config=encoding_config)
PrintDatetime()

In [ ]:
PrintDatetime("Started training")
hpt = {"learning_rate": [1e-2, 1e-3, 1e-4, 1e-5], "batch_size": [8, 32, 128]}
keys, values = zip(*hpt.items())
num_epoch = 1000
i = 1
for v in itertools.product(*values):
  model_save_path_base = "/content/models/HomemadeDataset/Base/{}".format(i)
  model_save_path_transformed = "/content/models/HomemadeDataset/Transformed/{}".format(i)
  model_save_path_cross = "/content/models/HomemadeDataset/Cross/{}".format(i)
  makedirs(model_save_path_base)
  makedirs(model_save_path_transformed)
  makedirs(model_save_path_cross)
  experiment = dict(zip(keys, v))
  print(experiment)
  learning_rate = experiment["learning_rate"]
  batch_size = experiment["batch_size"]

  PrintDatetime("Started")
  homemade_train_test_path = "{}/homemade_dataset_resized".format(train_test_path_base)
  full_dataset = pd.read_csv("{}/labels.csv".format(homemade_train_test_path))
  az_dataset = full_dataset[full_dataset["LabelEncoded"] <= 25]
  train_dataset, test_val_dataset = train_test_split(az_dataset, test_size=0.2, random_state=0, shuffle=True)
  test_dataset, val_dataset = train_test_split(test_val_dataset, test_size=0.5, random_state=0, shuffle=True)
  train_dataset.to_csv("{}/train_az.csv".format(homemade_train_test_path))
  test_dataset.to_csv("{}/test_az.csv".format(homemade_train_test_path))
  val_dataset.to_csv("{}/val_az.csv".format(homemade_train_test_path))
  main("{}/train_az.csv".format(homemade_train_test_path), "{}/test_az.csv".format(homemade_train_test_path), "{}/val_az.csv".format(homemade_train_test_path), learning_rate=learning_rate, batch_size=batch_size, num_epochs=num_epoch, model_save_path=model_save_path_base)
  PrintDatetime()

  homemade_train_test_path_transformed = "{}/homemade_dataset_transformed".format(train_test_path_base)
  full_dataset = pd.read_csv("{}/labels.csv".format(homemade_train_test_path_transformed))
  az_dataset = full_dataset[full_dataset["LabelEncoded"] <= 25]
  train_dataset, test_val_dataset = train_test_split(az_dataset, test_size=0.2, random_state=0, shuffle=True)
  test_dataset, val_dataset = train_test_split(test_val_dataset, test_size=0.5, random_state=0, shuffle=True)
  train_dataset.to_csv("{}/train_az.csv".format(homemade_train_test_path_transformed))
  test_dataset.to_csv("{}/test_az.csv".format(homemade_train_test_path_transformed))
  val_dataset.to_csv("{}/val_az.csv".format(homemade_train_test_path_transformed))
  PrintDatetime("Started training experiment")
  main("{}/train_az.csv".format(homemade_train_test_path_transformed), "{}/test_az.csv".format(homemade_train_test_path_transformed), "{}/val_az.csv".format(homemade_train_test_path_transformed), learning_rate=learning_rate, batch_size=batch_size, num_epochs=num_epoch, model_save_path=model_save_path_transformed)
  PrintDatetime()

  PrintDatetime("Started training experiment")
  main("{}/train_az.csv".format(homemade_train_test_path), "{}/test_az.csv".format(homemade_train_test_path_transformed), "{}/val_az.csv".format(homemade_train_test_path_transformed), learning_rate=learning_rate, batch_size=batch_size, num_epochs=num_epoch, model_save_path=model_save_path_cross)
  PrintDatetime()


  i += 1

PrintDatetime()